In [1]:
# Dependencies
! pip install census
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
#import gmaps

# Census API Keys
from config import (api_key)
from config import gkey
c = Census(api_key, year=2018)

In [6]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count

census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E", "B23025_005E", "B03001_003E","B02001_003E",
                          "B02001_005E","B06009_005E","B06009_006E"), {'for': 'zip code tabulation area:*'})


# Convert to DataFrame

census_df = pd.DataFrame(census_data)

# Column Reordering
census_df = census_df.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "B03001_003E": "Number of Hispanic Residents",
                                      "B02001_003E": "Number of African American Residents",
                                      "B02001_005E": "Number of Asian Residents",
                                      "B06009_005E": "Number of Residents w/Bachelor's Degree",
                                      "B06009_006E": "Number of Residents w/Graduate Degree",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_df["Poverty Rate"] = census_df["Poverty Count"]/census_df["Population"]*100

# Add in Employment Rate (Employment Count / Population)
census_df["Unemployment Rate"] = census_df["Unemployment Count"]/census_df["Population"]*100


# Final DataFrame
census_df = census_df.loc[(census_df['Zipcode'] == "20007")|(census_df['Zipcode'] == "20016")|
                         (census_df['Zipcode'] == "20008")| (census_df['Zipcode'] == "20009")| (census_df['Zipcode'] == "20010")
                         |(census_df['Zipcode'] == "20011")| (census_df['Zipcode'] == "20012")| (census_df['Zipcode'] == "20013")
                         |(census_df['Zipcode'] == "20390")| (census_df['Zipcode'] == "20015")| (census_df['Zipcode'] == "20373")
                         | (census_df['Zipcode'] == "20319")| (census_df['Zipcode'] == "20317")| (census_df['Zipcode'] == "20064")
                         | (census_df['Zipcode'] == "20057")| (census_df['Zipcode'] == "20052")| (census_df['Zipcode'] == "20037")
                         | (census_df['Zipcode'] == "20036")| (census_df['Zipcode'] == "20032")| (census_df['Zipcode'] == "20024")
                         | (census_df['Zipcode'] == "20020")| (census_df['Zipcode'] == "20019")| (census_df['Zipcode'] == "20018")
                         | (census_df['Zipcode'] == "20017")| (census_df['Zipcode'] == "20001")| (census_df['Zipcode'] == "20002")
                         | (census_df['Zipcode'] == "20003")| (census_df['Zipcode'] == "20004")| (census_df['Zipcode'] == "20005")
                         | (census_df['Zipcode'] == "20006")]
census_df
census_df['Coords']= ''
census_df

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Unemployment Count,Number of Hispanic Residents,Number of African American Residents,Number of Asian Residents,Number of Residents w/Bachelor's Degree,Number of Residents w/Graduate Degree,Zipcode,Poverty Rate,Unemployment Rate,Coords
6216,ZCTA5 20001,109318.0,46716.0,31.4,60678.0,6468.0,1597.0,4177.0,19287.0,3327.0,10752.0,12669.0,20001,13.845363,3.418529,
6217,ZCTA5 20002,84892.0,64895.0,33.2,52141.0,9269.0,2351.0,4103.0,30838.0,2018.0,13985.0,15075.0,20002,14.283073,3.622775,
6218,ZCTA5 20003,126323.0,31213.0,33.9,66331.0,2888.0,975.0,1873.0,8682.0,866.0,8403.0,10347.0,20003,9.252555,3.123698,
6219,ZCTA5 20004,147522.0,1975.0,40.7,109468.0,108.0,69.0,246.0,98.0,238.0,530.0,1094.0,20004,5.468354,3.493671,
6220,ZCTA5 20005,94611.0,13222.0,33.7,78166.0,1333.0,275.0,2486.0,2150.0,1129.0,3164.0,5235.0,20005,10.081682,2.079867,
6221,ZCTA5 20006,46319.0,2756.0,19.7,15576.0,275.0,109.0,285.0,312.0,384.0,151.0,233.0,20006,9.978229,3.955007,
6222,ZCTA5 20007,130920.0,27562.0,34.6,88413.0,3134.0,746.0,3076.0,1508.0,1911.0,6339.0,10710.0,20007,11.370728,2.706625,
6223,ZCTA5 20008,117195.0,29457.0,38.3,99315.0,1804.0,625.0,3753.0,2334.0,1617.0,7378.0,14134.0,20008,6.124181,2.121737,
6224,ZCTA5 20009,107128.0,52117.0,33.6,77556.0,6004.0,1593.0,8003.0,9823.0,3063.0,14259.0,20598.0,20009,11.520233,3.056584,
6225,ZCTA5 20010,88463.0,34713.0,32.3,49681.0,4613.0,1375.0,10161.0,8756.0,1613.0,6979.0,8691.0,20010,13.288970,3.961052,


In [3]:
census_df["Percent Minority Population"] = (census_df["Number of Asian Residents"]+
                                            census_df["Number of Hispanic Residents"]+
                                            census_df["Number of African American Residents"])/census_df["Population"]*100
census_df["Percent of Residents w/Bachelor's Degree or Higher"] = (census_df["Number of Residents w/Bachelor's Degree"]+
                                                                   census_df["Number of Residents w/Graduate Degree"])/census_df["Population"]*100
census_final = census_df[["Zipcode", "Population","Per Capita Income", "Poverty Rate", "Percent of Residents w/Bachelor's Degree or Higher", "Percent Minority Population"]]
census_final["Zipcode"] = census_final["Zipcode"].astype(int)
census_final

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Zipcode,Population,Per Capita Income,Poverty Rate,Percent of Residents w/Bachelor's Degree or Higher,Percent Minority Population
6216,20001,46716.0,60678.0,13.845363,50.134857,57.348660
6217,20002,64895.0,52141.0,14.283073,44.780029,56.951999
6218,20003,31213.0,66331.0,9.252555,60.071124,36.590523
6219,20004,1975.0,109468.0,5.468354,82.227848,29.468354
6220,20005,13222.0,78166.0,10.081682,63.522916,43.601573
6221,20006,2756.0,15576.0,9.978229,13.933237,35.595065
6222,20007,27562.0,88413.0,11.370728,61.856904,23.565053
6223,20008,29457.0,99315.0,6.124181,73.028482,26.153376
6224,20009,52117.0,77556.0,11.520233,66.882207,40.080972
6225,20010,34713.0,49681.0,13.288970,45.141590,59.142108


In [4]:
# Finding lat/lang so we can get a nearby address.

geocode_url = 'https://maps.googleapis.com/maps/api/geocode/json'


for index,row in census_df.iterrows():
    location = row['Zipcode']
    params = {
    'address': location,
    'key': gkey
    }
    response = requests.get(geocode_url, params=params)
    response = response.json()

    lat = response["results"][0]["geometry"]["location"]["lat"]
    lng = response["results"][0]["geometry"]["location"]["lng"]
    
    origin_latlng = f"{lat},{lng}"
    census_df.loc[index, 'Coords'] = origin_latlng

census_df.head()

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Unemployment Count,Number of Hispanic Residents,Number of African American Residents,Number of Asian Residents,Number of Residents w/Bachelor's Degree,Number of Residents w/Graduate Degree,Zipcode,Poverty Rate,Unemployment Rate,Coords,Percent Minority Population,Percent of Residents w/Bachelor's Degree or Higher
6216,ZCTA5 20001,109318.0,46716.0,31.4,60678.0,6468.0,1597.0,4177.0,19287.0,3327.0,10752.0,12669.0,20001,13.845363,3.418529,"38.912068,-77.0190228",57.348660,50.134857
6217,ZCTA5 20002,84892.0,64895.0,33.2,52141.0,9269.0,2351.0,4103.0,30838.0,2018.0,13985.0,15075.0,20002,14.283073,3.622775,"38.9052763,-76.98158769999999",56.951999,44.780029
6218,ZCTA5 20003,126323.0,31213.0,33.9,66331.0,2888.0,975.0,1873.0,8682.0,866.0,8403.0,10347.0,20003,9.252555,3.123698,"38.8831372,-76.9902276",36.590523,60.071124
6219,ZCTA5 20004,147522.0,1975.0,40.7,109468.0,108.0,69.0,246.0,98.0,238.0,530.0,1094.0,20004,5.468354,3.493671,"38.8951382,-77.0226217",29.468354,82.227848
6220,ZCTA5 20005,94611.0,13222.0,33.7,78166.0,1333.0,275.0,2486.0,2150.0,1129.0,3164.0,5235.0,20005,10.081682,2.079867,"38.908229,-77.03053899999999",43.601573,63.522916


In [5]:
census_df.sort_values("Household Income")

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Unemployment Count,Number of Hispanic Residents,Number of African American Residents,Number of Asian Residents,Number of Residents w/Bachelor's Degree,Number of Residents w/Graduate Degree,Zipcode,Poverty Rate,Unemployment Rate,Coords,Percent Minority Population,Percent of Residents w/Bachelor's Degree or Higher
6301,ZCTA5 20390,-666666666.0,359.0,21.2,21819.0,0.0,0.0,62.0,40.0,0.0,13.0,0.0,20390,0.000000,0.000000,"38.880052,-76.9941873",28.412256,3.621170
6299,ZCTA5 20319,-666666666.0,42.0,21.7,19436.0,0.0,0.0,3.0,6.0,13.0,13.0,0.0,20319,0.000000,0.000000,"38.8669531,-77.01758319999999",52.380952,30.952381
6242,ZCTA5 20064,-666666666.0,2207.0,19.9,4099.0,113.0,73.0,237.0,278.0,160.0,17.0,81.0,20064,5.120072,3.307657,"38.9369141,-77.0003067",30.584504,4.440417
6241,ZCTA5 20057,-666666666.0,3841.0,19.8,3915.0,11.0,79.0,384.0,356.0,696.0,0.0,48.0,20057,0.286384,2.056756,"38.9079399,-77.0717165",37.386097,1.249675
6239,ZCTA5 20052,-666666666.0,751.0,20.2,4226.0,0.0,24.0,38.0,79.0,79.0,26.0,0.0,20052,0.000000,3.195739,"38.8993373,-77.0485308",26.098535,3.462051
6300,ZCTA5 20373,-666666666.0,144.0,21.9,23941.0,0.0,0.0,43.0,10.0,0.0,5.0,0.0,20373,0.000000,0.000000,"38.858595,-77.01182460000001",36.805556,3.472222
6298,ZCTA5 20317,28077.0,406.0,86.0,28115.0,120.0,0.0,5.0,102.0,0.0,88.0,78.0,20317,29.556650,0.000000,"38.92,-76.99",26.354680,40.886700
6233,ZCTA5 20020,36075.0,56131.0,33.4,24722.0,18941.0,3466.0,1984.0,51268.0,515.0,3835.0,3453.0,20020,33.744277,6.174841,"38.8628755,-76.97006689999999",95.788424,12.983913
6232,ZCTA5 20019,36398.0,62625.0,34.1,23072.0,17222.0,4875.0,1784.0,59657.0,118.0,3793.0,2381.0,20019,27.500200,7.784431,"38.8910932,-76.94702199999999",98.297804,9.858683
6235,ZCTA5 20032,37711.0,42930.0,29.9,21379.0,12441.0,3729.0,1484.0,37713.0,142.0,2520.0,1930.0,20032,28.979734,8.686233,"38.8372336,-77.01614359999999",91.635220,10.365712
